In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import shutil
import gc
import zipfile
import matplotlib.pyplot as plt
from tqdm import tqdm
from google.colab import drive
from sklearn.model_selection import train_test_split

drive.mount('/tmp/gdrive')

local_zip = '/tmp/gdrive/MyDrive/Halitra/bisindo.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Mounted at /tmp/gdrive


In [ ]:
training_dir = os.path.join('/tmp', 'bisindo', 'images')
!mkdir '/tmp/validation'

validation_dataset_portion = 0.1

for class_dir_name in os.listdir(training_dir):
  class_dir_path = os.path.join(training_dir, class_dir_name)
  train_dataset_num = len(os.listdir(class_dir_path))

  val_dataset_num = train_dataset_num * validation_dataset_portion

  val_class_dir_path = os.path.join('/tmp/validation/', class_dir_name)
  os.mkdir(val_class_dir_path)

  counter = 0

  for file_name in os.listdir(class_dir_path):
    filepath = os.path.join(class_dir_path, file_name)
    if counter < val_dataset_num:
      destination = os.path.join(val_class_dir_path, file_name)
      shutil.move(filepath, destination)
      counter += 1

training = tf.keras.preprocessing.image.ImageDataGenerator (
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
validation = tf.keras.preprocessing.image.ImageDataGenerator (
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

training_generator = training.flow_from_directory(
    training_dir,
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True
)

validation_generator = validation.flow_from_directory(
    '/tmp/validation',
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True
)

model = tf.keras.models.Sequential ([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax')
])

model.compile (
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    training_generator,
    validation_data = validation_generator,
    epochs = 15
)

Found 10315 images belonging to 26 classes.
Found 1155 images belonging to 26 classes.
Epoch 1/15
323/323 [==============================] - 113s 331ms/step - loss: 3.1863 - accuracy: 0.0791 - val_loss: 2.7237 - val_accuracy: 0.2268
Epoch 2/15
323/323 [==============================] - 122s 377ms/step - loss: 2.0790 - accuracy: 0.4035 - val_loss: 1.5067 - val_accuracy: 0.5758
Epoch 3/15
323/323 [==============================] - 107s 331ms/step - loss: 1.3702 - accuracy: 0.6101 - val_loss: 1.1763 - val_accuracy: 0.6450
Epoch 4/15
323/323 [==============================] - 108s 333ms/step - loss: 1.1337 - accuracy: 0.6713 - val_loss: 1.0031 - val_accuracy: 0.7195
Epoch 5/15
323/323 [==============================] - 107s 330ms/step - loss: 0.9971 - accuracy: 0.7091 - val_loss: 0.9095 - val_accuracy: 0.7281
Epoch 6/15
323/323 [==============================] - 107s 333ms/step - loss: 0.9216 - accuracy: 0.7327 - val_loss: 0.8575 - val_accuracy: 0.7498
Epoch 7/15
323/323 [=================

In [ ]:
from pathlib import Path

model.save('/tmp/gdrive/MyDrive/Halitra/sign 1.h5')

my_model = tf.keras.models.load_model('/tmp/gdrive/MyDrive/Halitra/sign 1.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(my_model)
tflite_model = converter.convert()

tflite_path = '/tmp/gdrive/MyDrive/Halitra/sign_1.tflite'
tflite_path = Path(tflite_path)

# Simpan model TFLite ke file
tflite_path.write_bytes(tflite_model)

# Tampilkan konfigurasi model Keras
print(my_model.get_config())

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


{'name': 'sequential', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 150, 150, 3), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'conv2d_input'}, 'registered_name': None}, {'module': 'keras.layers', 'class_name': 'Conv2D', 'config': {'name': 'conv2d', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 150, 150, 3), 'filters': 32, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}, 'registered_na